In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
df = pd.read_json("data/world_bank_projects.json")

List column names

In [3]:
list(df)

['_id',
 'approvalfy',
 'board_approval_month',
 'boardapprovaldate',
 'borrower',
 'closingdate',
 'country_namecode',
 'countrycode',
 'countryname',
 'countryshortname',
 'docty',
 'envassesmentcategorycode',
 'grantamt',
 'ibrdcommamt',
 'id',
 'idacommamt',
 'impagency',
 'lendinginstr',
 'lendinginstrtype',
 'lendprojectcost',
 'majorsector_percent',
 'mjsector_namecode',
 'mjtheme',
 'mjtheme_namecode',
 'mjthemecode',
 'prodline',
 'prodlinetext',
 'productlinetype',
 'project_abstract',
 'project_name',
 'projectdocs',
 'projectfinancialtype',
 'projectstatusdisplay',
 'regionname',
 'sector',
 'sector1',
 'sector2',
 'sector3',
 'sector4',
 'sector_namecode',
 'sectorcode',
 'source',
 'status',
 'supplementprojectflg',
 'theme1',
 'theme_namecode',
 'themecode',
 'totalamt',
 'totalcommamt',
 'url']

# 1. Find top 10 countries with most projects

In [4]:
df.countryname.value_counts().head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
Nepal                              12
People's Republic of Bangladesh    12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64

# 2. Find top 10 themes

Create theme data frame that lists only the theme and code of each project

In [5]:
tdf = pd.DataFrame(columns = ["code","name"])
for k in df.mjtheme_namecode:
    tdf=tdf.append(json_normalize(k))
tdf = tdf.reset_index(drop="True")

In [6]:
tdf.name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

# 3. Fill in missing theme names

Create theme dict with code number as keys

In [7]:
themes = tdf[tdf.name != ""].drop_duplicates().set_index("code").T.to_dict('list')
theme_dict = {}
for k in themes.keys():
    c = k
    t = themes[k][0]
    theme_dict[c]=t

In [8]:
theme_dict

{'1': 'Economic management',
 '10': 'Rural development',
 '11': 'Environment and natural resources management',
 '2': 'Public sector governance',
 '3': 'Rule of law',
 '4': 'Financial and private sector development',
 '5': 'Trade and integration',
 '6': 'Social protection and risk management',
 '7': 'Social dev/gender/inclusion',
 '8': 'Human development',
 '9': 'Urban development'}

In [9]:
fill_df = tdf.copy()
for k in fill_df.itertuples():
    if k.name == "":
        fill_df.set_value(k[0],'name',theme_dict[k.code])

Test that blank names were replaced

In [10]:
tdf.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [11]:
fill_df.head()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


Redo part 2 with blank names filled in

In [12]:
fill_df.name.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64